In [1]:
import torch.nn as nn
import torch


In [2]:
torch.manual_seed(42)

In [40]:
def create_special_sparse_matrix(input_list, em_size):
    # Create an empty square matrix of size A x A
    square_matrix = torch.zeros(len(input_list), em_size)

    # Iterate through each row of the matrix
    for i in range(len(input_list)):
    # Set the element at column index B[i] to 1
        square_matrix[i][B[i]] = 1
        
    return square_matrix


## Creating Embedding Wights

In [4]:
sequence_len = 3
dictionary_size = 5
embedding_size = 2
input_tensor = [2,0,1]

In [5]:
class MyNeuralNetwork(nn.Module):
    def __init__(self, dictionary_size, embedding_size):
        super(MyNeuralNetwork, self).__init__()
        self.embedding = nn.Embedding(dictionary_size, embedding_size)
        self.linear1 = nn.Linear(6, 1)
        self.linear2 = nn.Linear(1, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.embedding(x)
        x = x.reshape(1,6)
        x = self.linear1(x)
        x = self.linear2(x)
        x = self.sigmoid(x)
        return x


In [6]:
embedding_weights = torch.randn((dictionary_size,embedding_size), requires_grad=True)

In [7]:
embedding_layer = nn.Embedding(dictionary_size,embedding_size)
embedding_layer.weight = torch.nn.Parameter(embedding_weights)
W0_CLONE = embedding_layer.weight.clone()
model = MyNeuralNetwork(dictionary_size,embedding_size)
model.embedding.weight = torch.nn.Parameter(embedding_weights)
target = torch.tensor([[1.]]).reshape((1,1))
A0 = torch.tensor([2,0,1])
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
output = model(A0)
loss = loss_fn(output,target)
loss.backward()

### Get model's weights

In [8]:
A0_E = embedding_layer(A0)
A0_E = A0_E.reshape(1,6)
W1 = model.linear1.weight
B1 = model.linear1.bias
W2 = model.linear2.weight
B2 = model.linear2.bias

### Get model's gradients

In [9]:
DW0_ = model.embedding.weight.grad
DW1_ = model.linear1.weight.grad
DB1_ = model.linear1.bias.grad
DW2_ = model.linear2.weight.grad
DB2_ = model.linear2.bias.grad

### Forward Propagation

In [12]:
sigmoid = nn.Sigmoid()
Z1 = (torch.matmul(A0_E,W1.T) + B1.reshape((1,1)))
A1 = Z1
Z2 =  torch.matmul(A1,W2.T) + B2.reshape((1,1))
A2 = sigmoid(Z2)

### Layer Two backpropagation


In [13]:
sigmoid = nn.Sigmoid()
DA2 =  -(torch.divide(target, A2) - torch.divide(1 - target, 1 - A2))
DZ2 = DA2 * (sigmoid(Z2) * (1-sigmoid(Z2)))

In [14]:

DW2 = torch.matmul(DZ2.T,A1)/1
DB2 = torch.sum(DZ2 ,dim=0,keepdims=True)/1 
DA1 = torch.matmul(DZ2,W2)      
DZ1 = DA1

### Layer One Backpropagation

In [41]:

DW1 = torch.matmul(DZ1.T,A0_E)/1
DB1 = torch.sum(DZ1 ,dim=0,keepdims=True)/1 
DA0 = torch.matmul(DZ1,W1)      
DZ0 = DA0

### Embedding Layer backpropagation

In [42]:
DZ0 = DZ0.reshape(3,2)

In [43]:
sparse = create_special_sparse_matrix(input_tensor,dictionary_size)  # sample should be sparse
DW0 = torch.matmul(DZ0.T,sparse)
print(DW0.T)
print(DW0_)

tensor([[ 0.0520, -0.0580],
        [ 0.0516, -0.0351],
        [ 0.0106, -0.1150],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000]], grad_fn=<PermuteBackward0>)
tensor([[ 0.0520, -0.0580],
        [ 0.0516, -0.0351],
        [ 0.0106, -0.1150],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000]])
